# Q1. Downloading the data

Download the data for January and February 2023.

In [1]:
import requests

yellow_trip_jan2023_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
yellow_trip_feb2023_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

def download_file(url):
    filename = url.split('/')[-1]
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

In [2]:
download_file(yellow_trip_jan2023_url)
download_file(yellow_trip_feb2023_url)

Read the data for January. How many columns are there?

In [3]:
import polars as pl

pdf_yellow_trip_202301 = pl.read_parquet('yellow_tripdata_2023-01.parquet')

In [4]:
pdf_yellow_trip_202301.schema

OrderedDict([('VendorID', Int64),
             ('tpep_pickup_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('tpep_dropoff_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('passenger_count', Float64),
             ('trip_distance', Float64),
             ('RatecodeID', Float64),
             ('store_and_fwd_flag', String),
             ('PULocationID', Int64),
             ('DOLocationID', Int64),
             ('payment_type', Int64),
             ('fare_amount', Float64),
             ('extra', Float64),
             ('mta_tax', Float64),
             ('tip_amount', Float64),
             ('tolls_amount', Float64),
             ('improvement_surcharge', Float64),
             ('total_amount', Float64),
             ('congestion_surcharge', Float64),
             ('airport_fee', Float64)])

In [5]:
pdf_yellow_trip_202301.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.1,1.0,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0
1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.9,1.0,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25
2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0


In [6]:
pdf_yellow_trip_202301.shape

(3066766, 19)

In [7]:
print(f"The dataset has {pdf_yellow_trip_202301.shape[1]} columns and {pdf_yellow_trip_202301.shape[0]} rows")

The dataset has 19 columns and 3066766 rows


# Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

In [8]:
pdf_yellow_trip_202301 = pdf_yellow_trip_202301.with_columns(
    duration=(pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60
)

In [9]:
pdf_yellow_trip_202301.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.433333
2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.1,1.0,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.316667
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0,12.75
1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.9,1.0,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25,9.616667
2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0,10.833333


In [10]:
# df_yellow_trip_202301["duration"] = df_yellow_trip_202301["duration"].apply(lambda dt: (dt.total_seconds()/60))

What's the standard deviation of the trips duration in January?

In [11]:
pdf_yellow_trip_202301.select('duration').describe()

statistic,duration
str,f64
"""count""",3.066766e6
"""null_count""",0.0
"""mean""",15.668995
"""std""",42.594351
"""min""",-29.2
"""25%""",7.116667
"""50%""",11.516667
"""75%""",18.3
"""max""",10029.183333


In [12]:
pdf_yellow_trip_202301.select('duration').std().item()

42.59435124195457

In [13]:
duration_std = pdf_yellow_trip_202301.select('duration').std().item()
print(f"The standard deviation of the trips duration is {duration_std:.2f}")

The standard deviation of the trips duration is 42.59


# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [14]:
pdf_yellow_trip_202301.select('duration').plot.hist()

:Histogram   [duration]   (duration_count)

In [15]:
selected_yellow_trip_202301 = pdf_yellow_trip_202301.filter(
    (pl.col('duration') >= 1) & (pl.col('duration') <= 60)
)

In [16]:
selected_yellow_trip_202301.select('duration').plot.hist()

:Histogram   [duration]   (duration_count)

What fraction of the records left after you dropped the outliers?

In [17]:
rows_before_dropped = pdf_yellow_trip_202301.shape[0]
rows_after_dropped = selected_yellow_trip_202301.shape[0]

In [18]:
fraction = rows_after_dropped / rows_before_dropped

In [19]:
print(f"The fraction of the records after we dropped the outlier is {fraction*100}%")

The fraction of the records after we dropped the outlier is 98.1220282212598%


# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it

In [20]:
selected_yellow_trip_202301.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.433333
2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.1,1.0,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.316667
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0,12.75
1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.9,1.0,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25,9.616667
2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0,10.833333


In [21]:
selected_yellow_trip_202301 = selected_yellow_trip_202301.with_columns(
    pl.col('PULocationID').cast(pl.Utf8).alias('PULocationID'),
    pl.col('DOLocationID').cast(pl.Utf8).alias('DOLocationID')
)

Let's select `PULocationID`, `DOLocationID`, and `duration`, then convert to **pandas DataFrame** for processing with scikit-learn module.

In [22]:
train_yellow_trip_202301 = selected_yellow_trip_202301.select(['PULocationID', 'DOLocationID', 'duration']).to_pandas()

In [23]:
train_yellow_trip_202301.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009173 entries, 0 to 3009172
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   PULocationID  object 
 1   DOLocationID  object 
 2   duration      float64
dtypes: float64(1), object(2)
memory usage: 68.9+ MB


In [24]:
train_yellow_trip_202301.head()

,PULocationID,DOLocationID,duration
0,161,141,8.433333
1,43,237,6.316667
2,48,238,12.750000
3,138,7,9.616667
4,107,79,10.833333


In [25]:
categorical = ['PULocationID', 'DOLocationID']

In [26]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [27]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=True)

In [28]:
independent_features = train_yellow_trip_202301[categorical].to_dict(orient='records')

In [29]:
independent_features[:10]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'}]

In [30]:
X_train = dv.fit_transform(independent_features)

What's the dimensionality of this matrix (number of columns)?

In [31]:
dv_matrix_dimension = len(dv.feature_names_)

In [32]:
print(f"The dimensionality of the matrix is {dv_matrix_dimension}")

The dimensionality of the matrix is 515


# Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters

Let's try include the `DictVectorizer` into pipeline.

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [35]:
regression_pipeline = Pipeline([
    ('vectorizer', DictVectorizer(sparse=True)),
    ('linreg', LinearRegression())
]) 

**Note**: The input for this pipeline ins dict-like object.

In [36]:
train_yellow_trip_202301.head()

,PULocationID,DOLocationID,duration
0,161,141,8.433333
1,43,237,6.316667
2,48,238,12.750000
3,138,7,9.616667
4,107,79,10.833333


In [37]:
X_train = train_yellow_trip_202301.iloc[:, :-1]
y_train = train_yellow_trip_202301.iloc[:, -1]

In [38]:
regression_pipeline.fit(X_train.to_dict('records'), y_train)

Pipeline(steps=[('vectorizer', DictVectorizer()),
                ('linreg', LinearRegression())])

Calculate the RMSE of the model on the training data

In [39]:
y_pred_train = regression_pipeline.predict(X_train.to_dict('records'))

In [40]:
from sklearn.metrics import root_mean_squared_error

rmse_train = root_mean_squared_error(y_train, y_pred_train)

In [41]:
print(f"RMSE of the model on the training data is {rmse_train:.2f}")

RMSE of the model on the training data is 7.65


# Q6. Evaluating the model

In [42]:
test_yellow_trip_202302 = pl.read_parquet('yellow_tripdata_2023-02.parquet') \
    .with_columns(
        duration=(pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60
    ) \
    .filter(
        (pl.col('duration') >= 1) & (pl.col('duration') <= 60)
    ).with_columns(
        pl.col('PULocationID').cast(pl.Utf8).alias('PULocationID'),
        pl.col('DOLocationID').cast(pl.Utf8).alias('DOLocationID')
    ).select(
        ['PULocationID', 'DOLocationID', 'duration']
    ).to_pandas()

In [43]:
test_yellow_trip_202302.shape

(2855951, 3)

In [44]:
pl.read_parquet('yellow_tripdata_2023-02.parquet') \
    .with_columns(
        duration=(pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60
    ) \
    .filter(
        (pl.col('duration') >= 1) & (pl.col('duration') <= 60)
    ).with_columns(
        pl.col('PULocationID').cast(pl.Utf8).alias('PULocationID'),
        pl.col('DOLocationID').cast(pl.Utf8).alias('DOLocationID')
    ).select(
        ['PULocationID', 'DOLocationID', 'duration']
    ).select('duration').plot.hist()

:Histogram   [duration]   (duration_count)

In [45]:
test_yellow_trip_202302.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2855951 entries, 0 to 2855950
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   PULocationID  object 
 1   DOLocationID  object 
 2   duration      float64
dtypes: float64(1), object(2)
memory usage: 65.4+ MB


In [46]:
test_yellow_trip_202302.head()

,PULocationID,DOLocationID,duration
0,142,163,1.683333
1,132,26,32.083333
2,161,145,13.300000
3,148,236,14.633333
4,137,244,27.950000


In [47]:
X_test = test_yellow_trip_202302.iloc[:, :-1]
y_test = test_yellow_trip_202302.iloc[:, -1]

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [48]:
y_pred_test = regression_pipeline.predict(X_test.to_dict('records'))
rmse_test = root_mean_squared_error(y_test, y_pred_test)

In [49]:
print(f"The RMSE on validation is {rmse_test}")

The RMSE on validation is 7.81181653393919
